In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from konlpy.tag import Komoran

import os

from keybert import KeyBERT
import tqdm

c:\ProgramData\Anaconda3\envs\DINS\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
data = pd.read_csv('./Data/SBS.csv')
data = data.iloc[4282:, :]

data.head(1)

,title,headline,date,link,content,category,site
4282,"슈팅 0개로 침묵한 손흥민, '북런던 더비' 5∼6점대 평점",잉글랜드 프리미어리그 '북런던 더비'에서 골 침묵을 지킨 손흥민이 현지 매체로부터 ...,2022.10.01,https://news.sbs.co.kr//news/endPage.do?news_i...,잉글랜드 프리미어리그 '북런던 더비'에서 골 침묵을 지킨 손흥민(30·토트넘)이...,스포츠,sbs


In [4]:
stopwords = pd.read_csv('./Data/stopwords.csv').noun.tolist()

len(stopwords)

9588

In [45]:
# date 변경, link col_name 변경 및 저장

# data.rename(columns={'news_link' : 'link'}, inplace=True)

# data.date = data.date.str.split(' ').str[0]

# data.head(1)

# data.to_csv('./Data/SBS.csv', index=False)

In [26]:
# okt = Okt()

# contents_nouns = []

In [6]:
# contents_nouns.append(p.map(get_nouns, contents))

In [4]:
# for text in contents:
#     nouns = okt.nouns(text)
    
#     text_nouns = ''
#     for noun in nouns:
#         text_nouns += noun + ' '
    
#     contents_nouns.append(text_nouns)
#     print(text_nouns)

In [22]:
# contents = data.iloc[79573:].content

# len(contents)

252

In [4]:
# data_sep = data.iloc[78237:]

# len(data_sep)

1588

In [5]:
dates = data.date.unique()

len(dates)
# 2022.10.01 ~ 2022.10.30

30

In [6]:
categories = data.category.unique()

len(categories)
# '사회', '국제', '스포츠', '생활·문화', '경제', '정치', '연예'

7

In [7]:
data_date = []

for date in dates:
    data_date.append(data.groupby('date').get_group(date))

In [8]:
datas = []


for data in data_date:
    data_category = []
    for category in categories:
        try:
            data_category.append(data.groupby('category').get_group(category).content)
        except (KeyError):
            pass
    
    datas.append(data_category)

In [9]:
komoran = Komoran()

contents_nouns = []

In [10]:
datas_nouns = []

for data_date in datas:
    contents_nouns = []
    
    for contents in data_date:
        content_nouns = ''
        
        for text in contents:
            poses = komoran.pos(text)
        
            for pos in poses:
                if len(pos[0]) < 3:
                    continue
                
                if pos[-1] == 'NNP':
                    if pos[0] not in stopwords:
                        content_nouns += pos[0] + ' '
            
        contents_nouns.append(content_nouns)
        
    datas_nouns.append(contents_nouns)

In [25]:
# content_nouns = ''

# for text in contents:
#     text_nouns = okt.nouns(text)
    
#     for noun in text_nouns:
#         content_nouns += noun + ' '
    

# contents_nouns.append(content_nouns)

In [11]:
# model = KeyBERT('distilbert-base-nli-mean-tokens')

model = KeyBERT('bert-base-nli-mean-tokens')

In [12]:
datas_keywords = []

for data_date in datas_nouns:
    contents_keywords = []
    
    for content_nouns in data_date:
        contents_keywords.append(model.extract_keywords(content_nouns, keyphrase_ngram_range=(1,1), top_n=20, use_mmr=True))
    
    datas_keywords.append(contents_keywords)

In [13]:
result_keywords = []

date_idx = 0
category_idx = 0

for data_keywords in datas_keywords:
    for keywords in data_keywords:
        for keyword in keywords:
            result_keywords.append((dates[date_idx], categories[category_idx], ) + keyword) # (date, category, keyword, importance)
    
        category_idx += 1
            
    date_idx += 1
    category_idx = 0

In [14]:
result = pd.DataFrame(result_keywords, columns=['date', 'category', 'keyword', 'importance'])

result

,date,category,keyword,importance
0,2022.10.01,스포츠,한국프로축구연맹,0.8569
1,2022.10.01,스포츠,캘리포니아주,0.8492
2,2022.10.01,스포츠,올림픽공원,0.8453
3,2022.10.01,스포츠,한국시리즈,0.8377
4,2022.10.01,스포츠,샌디에이고,0.8336
...,...,...,...,...
3998,2022.10.30,연예,아시아,0.7698
3999,2022.10.30,연예,서비스,0.7658
4000,2022.10.30,연예,페스티벌,0.7560
4001,2022.10.30,연예,콘서트,0.7455


In [15]:
result.to_csv('./Data/keywords.csv', index=False)

In [16]:
import pymysql

In [17]:
keywords = pd.read_csv('./Data/keywords.csv')

keywords

,date,category,keyword,importance
0,2022.10.01,스포츠,한국프로축구연맹,0.8569
1,2022.10.01,스포츠,캘리포니아주,0.8492
2,2022.10.01,스포츠,올림픽공원,0.8453
3,2022.10.01,스포츠,한국시리즈,0.8377
4,2022.10.01,스포츠,샌디에이고,0.8336
...,...,...,...,...
3998,2022.10.30,연예,아시아,0.7698
3999,2022.10.30,연예,서비스,0.7658
4000,2022.10.30,연예,페스티벌,0.7560
4001,2022.10.30,연예,콘서트,0.7455


In [18]:
sql_insert = 'INSERT INTO keywords (date, category, keyword, importance) VALUES (%s, %s, %s, %s)'

In [19]:
conn = pymysql.connect(host='localhost', user='root', password='1234', db='dins', charset='utf8')
cursor = conn.cursor()


for index, row in keywords.iterrows():
    cursor.execute(sql_insert, (row.date, row.category, row.keyword, row.importance))

conn.commit()
conn.close()

In [106]:
# keywords = model.extract_keywords(contents_nouns, keyphrase_ngram_range=(1,1), top_n=15, use_mmr=True)

# keywords